In [6]:
import pandas as pd
import numpy as np

def run_full_preprocessing_silent(input_file_path, output_file_name='final_sleep_data_clean.csv'):
    try:
        df = pd.read_csv(input_file_path)

        df.columns = df.columns.str.replace(' ', '_').str.replace('[^A-Za-z0-9_]+', '', regex=True)
        
        if 'Blood_Pressure' in df.columns:
            df[['Systolic_BP_Raw', 'Diastolic_BP']] = df['Blood_Pressure'].astype(str).str.split('/', expand=True)
            df['Systolic_BP'] = pd.to_numeric(
                df['Systolic_BP_Raw'].str.replace('W ', '', regex=False).str.strip(),
                errors='coerce'
            )
            df['Diastolic_BP'] = pd.to_numeric(df['Diastolic_BP'].str.strip(), errors='coerce')
            df.drop(columns=['Blood_Pressure', 'Systolic_BP_Raw'], inplace=True)
        
        for col in ['Occupation', 'BMI_Category', 'Gender', 'Sleep_Disorder']:
            if col in df.columns:
                df[col] = df[col].astype(str).str.strip().str.title()
                df[col] = df[col].replace('Nan', np.nan) 
        
        numerical_cols = ['Age', 'Sleep_Duration', 'Quality_of_Sleep', 'Physical_Activity_Level', 
                          'Stress_Level', 'Heart_Rate', 'Daily_Steps', 'Systolic_BP', 'Diastolic_BP']
        for col in numerical_cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        columns_to_check = df.columns.drop('Person_ID').tolist()
        df = df.drop_duplicates(subset=columns_to_check, keep='first')
        
        df = df.reset_index(drop=True)
        df['Person_ID_New'] = df.index + 1
        df.drop(columns=['Person_ID'], inplace=True)
        df.rename(columns={'Person_ID_New': 'Person_ID'}, inplace=True)
        
        cols = ['Person_ID'] + [col for col in df.columns if col != 'Person_ID']
        df = df[cols]
        
        df.to_csv(output_file_name, index=False)
        
    except FileNotFoundError:
        print(f"ERROR: The file '{input_file_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred during processing: {e}")

input_file = 'Sleep_health_and_lifestyle_dataset.csv'
output_file = 'final_sleep_data_clean.csv'

run_full_preprocessing_silent(input_file, output_file)